# Search and Filter DataFrames in PySpark

Once we have created our Spark Session, read in the data we want to work with and done some basic validation, the next thing you'll want to do is start exploring your dataframe. There are several option in PySpark to do this, so we are going to start with the following in this lecture, and continue to dive deeper in the next several lectures. 

### Agenda:

 - Introduce PySparks SQL funtions library 
 - Select method
 - Order By
 - Like Operator (for searching a string)
 - Substring Search
 - Is In Operator
 - Starts with, Ends with
 - Slicing
 - Filtering
 - Collecting Results as Objects

Let's get started!

In [1]:
# First let's create our PySpark instance
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Select").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 16:56:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/18 16:56:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/18 16:56:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/11/18 16:56:02 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


You are working with 1 core(s)


## Read in the DataFrame for this Notebook

In [2]:
path = 'Datasets/'
fifa = spark.read.csv('Datasets/fifa19.csv',inferSchema=True,header=True)

## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below: 

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, 
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

In [3]:
# Take a look at the first few lines
fifa.limit(4).toPandas()

25/11/18 16:56:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/18 16:56:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Body Type, Real Face, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, HeadingAccuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle,

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68,15,21,13,90,85,87,88,94,€138.6M


## Very important for trunctate

In [4]:
# To display ALL columns
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Take a look at the first few lines
fifa.limit(4).toPandas()

25/11/18 16:57:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Body Type, Real Face, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, HeadingAccuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, Release Clause
 Schema: _c0, ID, Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value,

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5,4,4,Medium/ Medium,Messi,Yes,RF,10,"Jul 1, 2004",None,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5,4,5,High/ Low,C. Ronaldo,Yes,ST,7,"Jul 10, 2018",None,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,"Aug 3, 2017",None,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4,3,1,Medium/ Medium,Lean,Yes,GK,1,"Jul 1, 2011",None,2020,6'4,168lbs,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17,13,21,50,13,18,21,19,51,42,57,58,60,90,43,31,67,43,64,12,38,30,12,68,40,68,15,21,13,90,85,87,88,94,€138.6M


In [5]:
print(fifa.printSchema())

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

## Select
There are a variety of functions you can import from pyspark.sql.functions. Check out the documentation for the full list available:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [6]:
# Import the functions we will need:
from pyspark.sql.functions import *
# countDistinct,avg,stddev
# abs # Absolute value
# acos # inverse cosine of col, as if computed by java.lang.Math.acos()

Since this is a sql function, the calls are pretty intuitive....

In [7]:
fifa.select(['Nationality','Name','Age']).show(5)

+-----------+-----------------+---+
|Nationality|             Name|Age|
+-----------+-----------------+---+
|  Argentina|         L. Messi| 31|
|   Portugal|Cristiano Ronaldo| 33|
|     Brazil|        Neymar Jr| 26|
|      Spain|           De Gea| 27|
|    Belgium|     K. De Bruyne| 27|
+-----------+-----------------+---+
only showing top 5 rows


**Order By**

In [10]:
# who is the youngest player in the dataset?
fifa.select(['Nationality','Name','Age']).orderBy("Age").show(5)

+-----------+--------------+---+
|Nationality|          Name|Age|
+-----------+--------------+---+
|     Sweden|     B. Nygren| 16|
|     France|   W. Geubbels| 16|
|     Sweden|  H. Andersson| 16|
|    Belgium|Y. Verschaeren| 16|
|     Turkey|      A. Doğan| 16|
+-----------+--------------+---+
only showing top 5 rows


In [11]:
fifa.select(['Nationality','Name', 'Age']).orderBy(fifa['Age'].desc()).show(20)

+-----------------+---------------+---+
|      Nationality|           Name|Age|
+-----------------+---------------+---+
|           Mexico|       O. Pérez| 45|
|Trinidad & Tobago|      T. Warner| 44|
|          England|  K. Pilkington| 44|
|            Japan|    S. Narazaki| 42|
|         Paraguay|      J. Villar| 41|
|     Saudi Arabia|   H. Sulaimani| 41|
|           France|       B. Nivet| 41|
|          England|       M. Tyler| 41|
|        Argentina|       C. Muñoz| 41|
|          England|    S. Phillips| 40|
|            Italy|      G. Buffon| 40|
|         Colombia|    B. Castillo| 40|
|        Argentina|    A. Bizzarri| 40|
|           Norway|       F. Kippe| 40|
|           Brazil|         Hilton| 40|
|         Colombia|        W. Díaz| 40|
|        Argentina|     S. Bertoli| 40|
|            Japan|    Y. Nakazawa| 40|
|            Japan|    S. Nakamura| 40|
|      Netherlands|P. van der Vlag| 40|
+-----------------+---------------+---+
only showing top 20 rows


In [12]:
fifa.select(['Nationality', 'Name', 'Age']).orderBy(fifa['Nationality'].desc()).show()

+-----------+-------------+---+
|Nationality|         Name|Age|
+-----------+-------------+---+
|   Zimbabwe|   M. Nakamba| 24|
|   Zimbabwe|    T. Hadebe| 22|
|   Zimbabwe|   K. Billiat| 27|
|   Zimbabwe|  A. Chicksen| 26|
|   Zimbabwe|  W. Katsande| 32|
|   Zimbabwe|  T. Kadewere| 22|
|   Zimbabwe|  N. Mushekwi| 30|
|   Zimbabwe|   M. Munetsi| 22|
|   Zimbabwe|   T. Darikwa| 26|
|   Zimbabwe|    K. Musona| 28|
|   Zimbabwe|   K. Mahachi| 24|
|   Zimbabwe|C. Nhamoinesu| 32|
|   Zimbabwe|   Q. Antipas| 34|
|     Zambia|      P. Daka| 19|
|     Zambia|  E. Chilufya| 18|
|     Zambia|    F. Sakala| 21|
|     Zambia|     S. Sunzu| 29|
|     Zambia|     E. Mwepu| 20|
|     Zambia|   A. Mulenga| 28|
|     Zambia|     E. Banda| 20|
+-----------+-------------+---+
only showing top 20 rows


In [8]:
# Who is the oldest player?
fifa.select(['Nationality','Name','Age']).orderBy(fifa["Age"].desc()).show(5)

+-----------------+-------------+---+
|      Nationality|         Name|Age|
+-----------------+-------------+---+
|           Mexico|     O. Pérez| 45|
|Trinidad & Tobago|    T. Warner| 44|
|          England|K. Pilkington| 44|
|            Japan|  S. Narazaki| 42|
|         Paraguay|    J. Villar| 41|
+-----------------+-------------+---+
only showing top 5 rows


**Like**

In [9]:
# If we wanted to look for all players that had "Barcelona" in their club title 
# We could use the like operator
fifa.select("Name","Club").where(fifa.Club.like("%Barcelona%")).show(5, False)

+---------------+------------+
|Name           |Club        |
+---------------+------------+
|L. Messi       |FC Barcelona|
|L. Suárez      |FC Barcelona|
|M. ter Stegen  |FC Barcelona|
|Sergio Busquets|FC Barcelona|
|Coutinho       |FC Barcelona|
+---------------+------------+
only showing top 5 rows



**Substrings**

.substr(starting postion,length)

Use this if you want to return a particular portion within a string

In [10]:
# Select last 4 characters of the photo column to understand all file types used
# This says return 
fifa.select("Photo",fifa.Photo.substr(-4,4)).show(5,False)

+----------------------------------------------+-----------------------+
|Photo                                         |substring(Photo, -4, 4)|
+----------------------------------------------+-----------------------+
|https://cdn.sofifa.org/players/4/19/158023.png|.png                   |
|https://cdn.sofifa.org/players/4/19/20801.png |.png                   |
|https://cdn.sofifa.org/players/4/19/190871.png|.png                   |
|https://cdn.sofifa.org/players/4/19/193080.png|.png                   |
|https://cdn.sofifa.org/players/4/19/192985.png|.png                   |
+----------------------------------------------+-----------------------+
only showing top 5 rows



In [11]:
# Or we could get the date that the string of numbers there
fifa.select("Photo",fifa.Photo.substr(32, 11)).show(5,False)

+----------------------------------------------+------------------------+
|Photo                                         |substring(Photo, 32, 11)|
+----------------------------------------------+------------------------+
|https://cdn.sofifa.org/players/4/19/158023.png|4/19/158023             |
|https://cdn.sofifa.org/players/4/19/20801.png |4/19/20801.             |
|https://cdn.sofifa.org/players/4/19/190871.png|4/19/190871             |
|https://cdn.sofifa.org/players/4/19/193080.png|4/19/193080             |
|https://cdn.sofifa.org/players/4/19/192985.png|4/19/192985             |
+----------------------------------------------+------------------------+
only showing top 5 rows



**ISIN**

You can also use ISIN to search for a list of options within a column.

In [16]:
fifa[fifa.Nationality.isin("Portugal","Argentina")].limit(10).toPandas()

25/11/18 17:14:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , ID, Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Body Type, Real Face, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, HeadingAccuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, Release Clause
 Schema: _c0, ID, Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value,

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5,4,4,Medium/ Medium,Messi,Yes,RF,10,"Jul 1, 2004",None,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5,4,5,High/ Low,C. Ronaldo,Yes,ST,7,"Jul 10, 2018",None,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23,7,11,15,14,11,€127.1M
2,15,211110,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€89M,€205K,2092,Left,3,3,4,High/ Medium,Normal,Yes,LF,21,"Jul 1, 2015",None,2022,5'10,165lbs,83+3,83+3,83+3,87+3,86+3,86+3,86+3,87+3,87+3,87+3,87+3,86+3,79+3,79+3,79+3,86+3,62+3,58+3,58+3,58+3,62+3,56+3,45+3,45+3,45+3,56+3,82,84,68,87,88,92,88,88,75,92,87,83,91,86,85,82,75,80,65,88,48,32,84,87,86,84,23,20,20,5,4,4,5,8,€153.5M
3,23,153079,S. Agüero,30,https://cdn.sofifa.org/players/4/19/153079.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,89,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€64.5M,€300K,2107,Right,4,4,4,High/ Medium,Stocky,Yes,ST,10,"Jul 28, 2011",None,2021,5'8,154lbs,86+3,86+3,86+3,86+3,87+3,87+3,87+3,86+3,85+3,85+3,85+3,83+3,76+3,76+3,76+3,83+3,58+3,56+3,56+3,56+3,58+3,53+3,47+3,47+3,47+3,53+3,70,93,77,81,85,89,82,73,64,89,88,80,86,90,91,88,81,76,73,83,65,24,92,83,83,90,30,20,12,13,15,6,11,14,€119.3M
4,38,167664,G. Higuaín,30,https://cdn.sofifa.org/players/4/19/167664.png,Argentina,https://cdn.sofifa.org/flags/52.png,88,88,Milan,https://cdn.sofifa.org/teams/2/light/47.png,€57M,€245K,1965,Right,4,4,3,High/ Medium,Normal,Yes,LS,9,None,Juventus,"Jun 30, 2019",6'1,196lbs,85+3,85+3,85+3,81+3,83+3,83+3,83+3,81+3,80+3,80+3,80+3,78+3,71+3,71+3,71+3,78+3,55+3,53+3,53+3,53+3,55+3,51+3,48+3,48+3,48+3,51+3,68,92,80,75,90,84,74,62,59,85,73,73,75,86,69,86,79,70,85,80,50,20,92,74,70,86,35,22,18,5,12,7,5,10,None
5,43,201399,M. Icardi,25,https://cdn.sofifa.org/players/4/19/201399.png,Argentina,https://cdn.sofifa.org/flags/52.png,87,90,Inter,https://cdn.sofifa.org/teams/2/light/44.png,€64.5M,€130K,1940,Right,3,4,3,Medium/ Low,Normal,Yes,ST,9,"Jul 9, 2013",None,2021,5'11,165lbs,84+3,84+3,84+3,76+3,81+3,81+3,81+3,76+3,76+3,76+3,76+3,74+3,68+3,68+3,68+3,74+3,54+3,53+3,53+3,53+3,54+3,52+3,51+3,51+3,51+3,52+3,46,91,91,73,85,77,58,45,54,81,77,78,76,88,76,84,94,72,76,70,56,22,92,64,82,85,45,24,20,13,9,5,7,9,€114.5M
6,56,218667,Bernardo Silva,23,https://cdn.sofifa.org/players/4/19/218667.png,Portugal,https://cdn.sofifa.org/flags/38.png,86,91,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€59.5M,€180K,2039,Left,2,

**Starts with Ends with**

Search for a specific case - begins with "x" and ends with "x"

In [18]:
fifa.select("Name","Club").where(fifa.Name.startswith("L")).where(fifa.Name.endswith("i")).limit(4).toPandas()

,Name,Club
0,L. Messi,FC Barcelona
1,L. Bonucci,Juventus
2,L. Fabiański,West Ham United
3,L. Pellegrini,Roma


#### Slicing a Dataframe

In [19]:
# Starting
print('Starting row count:',fifa.count())
print('Starting column count:',len(fifa.columns))

# Slice rows
df2 = fifa.limit(300)
print('Sliced row count:',df2.count())

# Slice columns
cols_list = fifa.columns[0:5]
df3 = fifa.select(cols_list)
print('Sliced column count:',len(df3.columns))

Starting row count: 18207
Starting column count: 89
Sliced row count: 300
Sliced column count: 5


**Slicing Method**

pyspark.sql.functions.slice(x, start, length)[source] <br>
Returns an array containing all the elements in x from index start (or starting from the end if start is negative) with the specified length.  <br>
<br>
*Note: indexing starts at 1 here*

In [9]:
# This is within an array
from pyspark.sql.functions import slice

df = spark.createDataFrame([([1, 2, 3],), ([4, 5],)], ['x']) 
df.show()
df.select(slice(df.x, 2, 2).alias("sliced")).show()

+---------+
|        x|
+---------+
|[1, 2, 3]|
|   [4, 5]|
+---------+

+------+
|sliced|
+------+
|[2, 3]|
|   [5]|
+------+



In [34]:
df = spark.createDataFrame([([1, 2, 3],[1, 2, 3],), ([4, 5],[1, 2, 3],)], ['x']) 
df.show()

+---------+---------+
|        x|       _2|
+---------+---------+
|[1, 2, 3]|[1, 2, 3]|
|   [4, 5]|[1, 2, 3]|
+---------+---------+



If we want to simply slice our dataframe (ie. limit the number of rows or columns) we can do this...

## Filtering Data

A large part of working with DataFrames is the ability to quickly filter out data based on conditions. Spark DataFrames are built on top of the Spark SQL platform, which means that is you already know SQL, you can quickly and easily grab that data using SQL commands, or using the DataFram methods (which is what we focus on in this course).

In [18]:
fifa.filter("Overall>50").limit(4).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68,15,21,13,90,85,87,88,94,€138.6M


In [20]:
# Using SQL with .select()
fifa.filter("Overall>50").select(['ID','Name','Nationality','Overall']).limit(4).toPandas()

,ID,Name,Nationality,Overall
0,158023,L. Messi,Argentina,94
1,20801,Cristiano Ronaldo,Portugal,94
2,190871,Neymar Jr,Brazil,92
3,193080,De Gea,Spain,91


**Try it yourself!**

Edit the line below to select only closing values above 800

In [22]:
# Try it yourself!
# Edit the line below to select only overall scores of LESS THAN 80
fifa.filter("Overall<80").select(['ID','Name','Nationality','Overall']).limit(4).toPandas()

,ID,Name,Nationality,Overall
0,244369,V. Tsygankov,Ukraine,79
1,239818,Rúben Dias,Portugal,79
2,236632,David Neres,Brazil,79
3,233419,Raphinha,Brazil,79


In [21]:
fifa.select(['Nationality','Name','Age','Overall']).filter("Overall>70").orderBy(fifa["Overall"].desc()).show()

+-----------+-----------------+---+-------+
|Nationality|             Name|Age|Overall|
+-----------+-----------------+---+-------+
|  Argentina|         L. Messi| 31|     94|
|   Portugal|Cristiano Ronaldo| 33|     94|
|     Brazil|        Neymar Jr| 26|     92|
|      Spain|           De Gea| 27|     91|
|    Belgium|     K. De Bruyne| 27|     91|
|    Belgium|        E. Hazard| 27|     91|
|    Croatia|        L. Modrić| 32|     91|
|    Uruguay|        L. Suárez| 31|     91|
|      Spain|     Sergio Ramos| 32|     91|
|   Slovenia|         J. Oblak| 25|     90|
|     Poland|   R. Lewandowski| 29|     90|
|    Germany|         T. Kroos| 28|     90|
|    Uruguay|         D. Godín| 32|     90|
|      Spain|      David Silva| 32|     90|
|     France|         N. Kanté| 27|     89|
|  Argentina|        P. Dybala| 24|     89|
|    England|          H. Kane| 24|     89|
|     France|     A. Griezmann| 27|     89|
|    Germany|    M. ter Stegen| 26|     89|
|    Belgium|      T. Courtois| 

### Collecting Results as Objects

The last thing we need to cover is collecting results as objects. If we wanted to say print individual names from an output, we need to essentially remove the item from the dataframe into an object. Like this

In [28]:
# Collecting results as Python objects
# you need the ".collect()" call at the end to "collect" the results
result = fifa.select(['Nationality','Name','Age','Overall']).filter("Overall>70").orderBy(fifa["Overall"].desc()).collect()

In [29]:
# Note the nested structure returns a nested row object
type(result[0])

pyspark.sql.types.Row

If we want to call on these results it would look something like this...

*Think of it like a matrix, first number is the row number and the second is the column number*

In [30]:
print("Best Player Over 70: ",result[0][1])
print("Nationality of Best Player Over 70: ",result[0][0])
print("")
print("Worst Player Over 70: ",result[-1][1])
print("Nationality of Worst Player Over 70: ",result[-1][0])

Best Player Over 70:  L. Messi
Nationality of Best Player Over 70:  Argentina

Worst Player Over 70:  Zapater
Nationality of Worst Player Over 70:  Spain


Rows can also be called to turn into dictionaries if needed

In [32]:
row.asDict()

NameError: name 'row' is not defined

Or iterated over like this...

In [34]:
for item in result[2]:
    print(item)

Brazil
Neymar Jr
26
92


Check out this link for more info on other methods:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

### Great job! That's it!